In [1]:
from tkinter import *
from tkinter import StringVar
from tkinter import scrolledtext
from tkinter import messagebox
import os,cv2;
import numpy as np
from PIL import Image
from xlutils.copy import copy
import xlwt 
import xlrd 
import time
import sys
from playsound import playsound



In [3]:
root = Tk()

def capture(root):
    iid = None  # type: Optional[StringVar]
    
    def assure_path_exists(path):
        dir = os.path.dirname(path)
        if not os.path.exists(dir):
            os.makedirs(dir)
    iid = IntVar()
    Label(root, text=" Enter your id  ", font="ariel 14 bold", bg='black', fg='white').place(x=650, y=200)
    Entry(root, textvariable=iid, bd=13).place(x=750, y=300)
    Button(root, text=" Done  ", font="ariel 20 bold", bg='black', fg='gold',command=lambda: mainwindow(root)).place(x=750, y=250)
    face_id= iid
    # Start capturing video 
    vid_cam = cv2.VideoCapture(0)

    # Detect object in video stream using Haarcascade Frontal Face
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    # Initialize sample face image
    count = 0

    assure_path_exists("dataset/")

    # Start looping
    while(True):

        # Capture video frame
        _, image_frame = vid_cam.read()

        # Convert frame to grayscale
        gray = cv2.cvtColor(image_frame, cv2.COLOR_BGR2GRAY)

        # Detect frames of different sizes, list of faces rectangles
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        # Loops for each faces
        for (x,y,w,h) in faces:

            # Crop the image frame into rectangle
            cv2.rectangle(image_frame, (x,y), (x+w,y+h), (255,0,0), 2)

            # Increment sample face image
            count += 1

            # Save the captured image into the datasets folder
            cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

            # Display the video frame, with bounded rectangle on the person's face
            cv2.imshow('frame', image_frame)

        # To stop taking video, press 'q' for at least 100ms
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

        # If image taken reach 40, stop taking video
        elif count>=40:
            print("Successfully Captured")
            break

    vid_cam.release()         # Stop video
    cv2.destroyAllWindows()   # Close all started windows

def train():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    detector= cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

    def getImagesAndLabels(path):
        #get the path of all the files in the folder
        imagePaths=[os.path.join(path,f) for f in os.listdir(path)]
        #create empth face list
        faceSamples=[]
        #create empty ID list
        Ids=[]
        #now looping through all the image paths and loading the Ids and the images
        for imagePath in imagePaths:
            #loading the image and converting it to gray scale
            pilImage=Image.open(imagePath).convert('L')
            #Now we are converting the PIL image into numpy array
            imageNp=np.array(pilImage,'uint8')
            #getting the Id from the image
            Id=int(os.path.split(imagePath)[-1].split(".")[1])
            split=(os.path.split(imagePath)[-1])
            # extract the face from the training image sample
            faces=detector.detectMultiScale(imageNp)
            #If a face is there then append that in the list as well as Id of it
            for (x,y,w,h) in faces:
                faceSamples.append(imageNp[y:y+h,x:x+w])
                Ids.append(Id)
        return faceSamples,Ids

    faces,Ids = getImagesAndLabels('dataSet')
    s = recognizer.train(faces, np.array(Ids))
    print("Successfully trained")
    recognizer.write('Trainer_trial/trainer.yml')

    
def recognition():
    # Workbook is opened
    rb = xlrd.open_workbook('Attendance_FileNew.xls')
    r_sheet = rb.sheet_by_index(0) 
    row = r_sheet.nrows
    wb = copy(rb) 
    sheet1 = wb.get_sheet(0) 
    # Reading all the entries from the excel sheet
    dict=[]
    print(row)
    if row>1:
        for r in range(1,row):
            cell=r_sheet.cell(r,1)
            dict.append(cell.value)

    start=time.time()
    period=8
    face_cas = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0);
    recognizer = cv2.face.LBPHFaceRecognizer_create();
    recognizer.read('Trainer_trial/trainer.yml');
    flag = 0;
    c1=c2=cc3=c5=0 #counting no. of correct guesses for 5 users

    #saving to file
    def save_to_file(id,n_id,row,dict):
        if(id not in dict):
            sheet1.write(row,0,n_id)
            sheet1.write(row,1,id)
            sheet1.write(row,2,"Yes") 
            dict.append(id);
            row+=1
        else:
            print(" ID : {} Sorry ! Already marked your Attendance!".format(n_id))
            cv2.putText(img,str(id)+" "+str(conf),(x,y-10),font,0.55,(120,255,120),1)
            cv2.putText(img," You've Been Marked! ",(x+10,y+220),font,0.55,(0,0,255),1)

    font = cv2.FONT_HERSHEY_SIMPLEX
    while True:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cas.detectMultiScale(gray, 1.3, 7)

        for (x,y,w,h) in faces:
            roi_gray = gray[y:y + h, x:x + w]
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),2)
            id,conf=recognizer.predict(roi_gray)
            if(conf < 50):
                if(id==1):
                    n_id=id
                    c1+=1
                    id='Crazy Rana'
                    if c1>3:
                        save_to_file(id,n_id,row,dict)

                elif(id==2):
                    n_id=id
                    c2+=1
                    id = 'Dolly Rana'
                    if c1>3:
                        save_to_file(id,n_id,row,dict)

                elif(id==3):
                    n_id=id
                    c3+=1
                    id = 'Himanshu Rana'
                    if c1>3:
                        save_to_file(id,n_id,row,dict)

                elif(id==4):
                    n_id=id
                    c4+=1
                    id = 'Tongue out'
                    if c4>3:
                        save_to_file(id,n_id,row,dict)

                elif(id==5):
                    n_id=id
                    c5+=1
                    id = "Peeru"
                    if c5>3:
                        save_to_file(id,n_id,row,dict)

            else:
                    print(" Sorry Can't Recognize your Face!")
                    flag=flag+1
                    break;


            cv2.putText(img,str(id)+" "+str(conf),(x,y-10),font,0.55,(120,255,120),1)
            #cv2.cv.PutText(cv2.cv.fromarray(img),str(id),(x,y+h),font,(0,0,255));
        cv2.imshow('frame',img);
        #cv2.imshow('gray',gray);

        if flag >= 15:
            flag="yes"
            playsound('transactionSound.mp3')
            print("Transaction Blocked")
            break;

        if time.time()>start+period:
            if flag!="yes":
                playsound('Sound.mp3')
            break;
        if cv2.waitKey(120) & 0xFF == ord('q'):
            if flag!="yes":
                playsound('Sound.mp3')
            break; 
    wb.save('Attendance_FileNew.xls')     
    cap.release();
    cv2.destroyAllWindows();



def quit(root) :  # WHEN USER HITS QUIT BUTTON
    root.destroy()  # root get destroys and tkinter window shuts down

def mainwindow(root):
    frame = Frame(root, width=1700, height=800, bg="black").place(x=0, y=0) # MINI WINDOW OF SMARTPHONES RECORD
    iframe = Frame(root, width=700, height=500, bg="white").place(x=400, y=150) # MINI WINDOW OF SMARTPHONES RECORD
    lbl = Label(root, text="Attendance Management System", bg='black', fg="gold", font=("arial bold", 60)).place(x=220, y=1)
    
    p1 = Button(frame, text=" Capture ", bg="black", fg="gold", font=("arial bold", 20), command=lambda: capture(root)).place(x=670, y=190)
    p1 = Button(root, text=" Train Data ", bg="black", fg="gold", font=("arial bold", 20),command=lambda: train()).place(x=650, y=290)
    p1 = Button(frame, text=" Recognition (marking attendance) ", bg="black", fg="gold", font=("arial bold", 20), command=lambda: recognition()).place(x=500, y=390)
    p1 = Button(frame, text=" Show Attendance sheet ", bg="black", fg="gold", font=("arial bold", 20),command=lambda: search(root)).place(x=585, y=490)
    p1 = Button(frame, text="   Exit  ", bg="black", fg="gold", font=("arial bold", 23),command=lambda: quit(root)).place(x=1330, y=730)

    
mainwindow(root)
root.title("Welcome to Priya's project")  # renames the title of the tkinter window
w, h = root.winfo_screenwidth(), root.winfo_screenheight()
root.geometry("%dx%d+0+0" % (w, h))  # this statement opens the tkinter window in maximized form
mainloop()

Successfully Captured
